<a href="https://colab.research.google.com/github/moh53/Graduation-Project-FMS/blob/NN-models/40MB/custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
data_X=np.load('drive/MyDrive/X.npy')
data_Y=np.load('drive/MyDrive/Y.npy')


In [ ]:
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import random
%matplotlib inline

In [ ]:
image_size=50
class IDCdataset(Dataset):
  def __init__(self, X, y):
        'Initialization'
        self.X = X
        self.y = y
  def __len__(self):
        'Denotes the total number of samples'
        return len(self.X)
  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        image = self.X[index]
        y = self.y[index]
        X = self.transform(image)
        #y = self.transform(label)
        return X ,y
  transform = T.Compose([
        T.ToPILImage(),
        T.Resize(image_size),
        T.ToTensor()])

In [ ]:
Dataset=IDCdataset(data_X,data_Y)


In [ ]:
print(len(Dataset))

5547


In [ ]:
print(Dataset[3759])

(tensor([[[0.5765, 0.4667, 0.5059,  ..., 0.5333, 0.7098, 0.6078],
         [0.6627, 0.6078, 0.5255,  ..., 0.5020, 0.5255, 0.7059],
         [0.4471, 0.5412, 0.5137,  ..., 0.5647, 0.6078, 0.6941],
         ...,
         [0.8118, 0.8314, 0.8235,  ..., 0.4392, 0.5725, 0.5333],
         [0.8824, 0.9216, 0.9294,  ..., 0.6078, 0.6980, 0.7294],
         [0.7137, 0.8824, 0.9176,  ..., 0.5412, 0.7059, 0.7725]],

        [[0.3922, 0.3216, 0.3412,  ..., 0.3961, 0.5451, 0.4431],
         [0.4431, 0.4549, 0.3608,  ..., 0.3569, 0.3843, 0.5020],
         [0.2667, 0.3608, 0.3490,  ..., 0.4118, 0.4549, 0.4902],
         ...,
         [0.5216, 0.5647, 0.5412,  ..., 0.2745, 0.3725, 0.3647],
         [0.7647, 0.8784, 0.8902,  ..., 0.4000, 0.4863, 0.5373],
         [0.3922, 0.7647, 0.9020,  ..., 0.3373, 0.3176, 0.3843]],

        [[0.5608, 0.4980, 0.5216,  ..., 0.5412, 0.6667, 0.5961],
         [0.6000, 0.6314, 0.5333,  ..., 0.5255, 0.5529, 0.6471],
         [0.4745, 0.5373, 0.5373,  ..., 0.5922, 0.6118, 0

In [ ]:
batch_size = 16
transformed_dataset = IDCdataset(data_X,data_Y)
train_dl = DataLoader(transformed_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=True)

In [ ]:
validation_split = .2
shuffle_dataset = True
random_seed= 1

In [ ]:
# Creating data indices for training and validation splits:
dataset_size = len(Dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
train_indices, val_indices = indices[split:], indices[:split]

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [ ]:
train_loader = torch.utils.data.DataLoader(Dataset, batch_size=batch_size,
                                           sampler=train_sampler,)
validation_loader = torch.utils.data.DataLoader(Dataset, batch_size=batch_size,
                                                sampler=valid_sampler)
print(len(train_loader))
print(len(validation_loader))

278
70


In [ ]:
import torch
from torch import nn
from torch import optim

In [ ]:
model = nn.Sequential(
    nn.Linear( 3 * 50 * 50 , 32),
    nn.ReLU(),
    nn.Linear(32 , 64),
    nn.ReLU(),
    nn.Linear(64 , 2)
)

In [ ]:
optimiser = optim.SGD(model.parameters(), lr=1e-2)

In [ ]:
loss = nn.BCEWithLogitsLoss()

In [ ]:
n_epochs = 15 
batch_size = 16
for epoch in range(n_epochs):
    losses = list()
    for batch in train_loader:
      'x is the images and the labels in the batch'
      x ,y = batch
      'reshape the image though they fit in the input NN as 3 * 50 * 50'
      x = x.view(x.size(0), -1)
      'x.shape[16, 7500] = [batch_size, 3*50*50]'
      'y.shape[16] indicate the label'
      
      l = model(x)
      'print(l.shape)-->[16,2]'
      
      'cuz the wants BCEWithLogitsLoss takes the l(logit) and y(target) be in the same shape  ' 
      j = loss(l[:,0], y.float())

      model.zero_grad()

      j.backward()

      optimiser.step()

      losses.append(j.item()
      )
    print(f'Epoch{epoch + 1}, train loss : {torch.tensor(losses).mean():.2f}')
       
    losses = list()

    for batch in validation_loader:
      x, y = batch
      
      'x = torch.div(x,255) if x is unit8 not float as expected '
      x = x.view(x.size(0), -1)

      with torch.no_grad():
          l = model(x)
      
      'cuz the wants BCEWithLogitsLoss takes the l(logit) and y(target) be in the same shape  ' 

      j = loss(l[:,0] ,y.float())
      
      losses.append(j.item())
    
    print(f'Epoch{epoch + 1}, validation loss : {torch.tensor(losses).mean():.2f}')




Epoch1, train loss : 0.51
Epoch1, validation loss : 1.04
Epoch2, train loss : 0.50
Epoch2, validation loss : 0.31
Epoch3, train loss : 0.51
Epoch3, validation loss : 1.35
Epoch4, train loss : 0.50
Epoch4, validation loss : 0.62
Epoch5, train loss : 0.49
Epoch5, validation loss : 0.53
Epoch6, train loss : 0.49
Epoch6, validation loss : 0.92
Epoch7, train loss : 0.49
Epoch7, validation loss : 0.85
Epoch8, train loss : 0.50
Epoch8, validation loss : 1.72
Epoch9, train loss : 0.49
Epoch9, validation loss : 1.55
Epoch10, train loss : 0.48
Epoch10, validation loss : 0.37
Epoch11, train loss : 0.49
Epoch11, validation loss : 0.93
Epoch12, train loss : 0.48
Epoch12, validation loss : 2.94
Epoch13, train loss : 0.49
Epoch13, validation loss : 0.84
Epoch14, train loss : 0.48
Epoch14, validation loss : 1.12
Epoch15, train loss : 0.48
Epoch15, validation loss : 0.64
